In [98]:
from typing import Dict, List, TypedDict
import spacy
from textacy import extract, preprocessing
from typing import Annotated
import operator
from langgraph.graph import StateGraph, START, END
from spacy.tokens import Doc
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
from pydantic import BaseModel, Field

In [99]:
load_dotenv()

#Danh sách từ không được tính là Actors
INTERNAL_SYSTEM_KEYWORDS = {"system", "software", "application", "app", "platform"}

model = init_chat_model(
  "gpt-5-mini",
  model_provider="openai",
) 

In [ ]:
class ValidationResult(BaseModel):
  is_valid: bool = Field(description="True if the data is accurate, False if it is not valid")
  reason: str = Field(description="The reason why the data is not valid")
  
class AliasItem(BaseModel):
  alias: str = Field(description="Name of the alias")
  sentences: List[int] = Field(description="List of sentence indices where THIS specific alias appears (starting from 1)")

class ActorAlias(BaseModel):
  actor: str = Field(description="The original actor's name")
  aliases: List[AliasItem] = Field(description="List of alternative names/references for this actor")
  
  def __str__(self) -> str:
    alias_str = "\n  ".join([f"- {item.alias}: sentences {item.sentences}" for item in self.aliases]) if self.aliases else "None"
    
    return (
        f"ACTOR: {self.actor}\n"
        f"  Aliases:\n  {alias_str}\n"
        f"{'-'*50}"
    )

class ActorList(BaseModel):
  actors: List[str] = Field(description="A list of actors who perform actions in the requirement.")
  
class ActorAliasMapping(BaseModel):
  mappings: List[ActorAlias] = Field(description="List of actor-alias mappings")

class UseCase(BaseModel):
  usecase: str = Field(description="The name of the use case (e.g., 'Place Order')")
  actor: str = Field(description="The actor who performs this use case")
  verb_phrase: str = Field(description="The extracted verb phrase")
  sentence: str = Field(description="The sentence where this use case was found")
  
  def __str__(self) -> str:
    return (
        f"USECASE: {self.usecase}\n"
        f"Actor: {self.actor}\n"
        f"Verb Phrase: {self.verb_phrase}\n"
        f"Sentence: {self.sentence}\n"
        f"{'-'*30}"
    )

In [101]:
# Khai báo State cho Graph
class GraphState(TypedDict):
  input_text: str     #Yêu cầu đầu vào của người dùng
  doc: Doc            #Doc sau khi gọi hàm nlp của SpaCy
  actors: List[str]   #Danh sách actors cuối cùng
  actor_aliases: List[ActorAlias] #Danh sách các alias của actor
  usecases: List[UseCase] #Danh sách use cases
  svo_elements: Annotated[List[Dict[str,str]], operator.add]
  validation_actor_result: ValidationResult

In [102]:
# actors node
def actors_node(state: GraphState):
  structured_llm = model.with_structured_output(ActorList)
  candidate_chunks = ", ".join(state["actors"])
  
  system_prompt = (
    "You are a Senior Systems Analyst and Linguistic Expert. Your task is to perform "
    "Entity Extraction specifically for 'Actors' in a system description or user story. "
    "An 'Actor' is defined as a person, organization, or external system that "
    "performs actions, initiates processes, or interacts with the system described."
)

  user_prompt = f"""
  I will provide you with a raw text and a list of potential 'Noun Chunks' extracted by a parser.

  ### RULES:
  1. **Filtering**: From the 'Candidate Noun Chunks', select only those that function as an active agent (Actor) in the 'Raw Text'.
  2. **Standardization**: Convert all extracted actors to their **singular form** (e.g., 'customers' -> 'customer').
  3. **Cleaning**: Remove any unnecessary articles (a, an, the) and honorifics.
  4. **Context Check**: Ensure the noun chunk is actually performing an action in the text, not just being mentioned as an object.
  5. **Exclude Self-References**: Do NOT include 'the system', 'the software', or 'the application' as an Actor if it refers to the system being described. These are internal components, not external actors.
  6. **External Systems**: Only include other specific systems if they are external entities that your system interacts with (e.g., 'Payment Gateway', 'External Database').
  
  ### INPUT DATA:
  - Raw Text: {state['input_text']}
  - Candidate Noun Chunks: {candidate_chunks}

  ### OUTPUT INSTRUCTIONS:
  Return only the final list of singularized actors.
  """
    
  response: ActorList = structured_llm.invoke([
    ("system", system_prompt),
    ("human", user_prompt)
  ])
  
  # print(response)
  
  return {
    "actors": response.actors
  }

In [103]:
# actors_alias_node

def actors_alias_node(state: GraphState):
  sentences = state.get("doc").sents
  
  structured_llm = model.with_structured_output(ActorAliasMapping)
  
  # Đầu vào cho LLMs
  input_sentences = [sentence.text.strip() for sentence in sentences]
  input_sentences = "\n".join([f"{i + 1}. {s}" for i, s in enumerate(input_sentences)])
  
  actors_list = ", ".join(state.get("actors"))
  
  system_prompt = (
    "You are an expert in Natural Language Processing and Entity Resolution. "
    "Your task is to identify all alternative references (aliases) to specific actors "
    "in a given text and map EACH UNIQUE ALIAS to the specific sentences where it appears."
  )
  
  user_prompt = f"""
  Analyze the following pre-segmented numbered sentences and identify all references to the given actors.

  ### ACTORS TO TRACK:
  {actors_list}

  ### PRE-SEGMENTED SENTENCES:
  {input_sentences}

  ### CRITICAL INSTRUCTIONS:
  - **Each unique alias must be tracked separately** with its own sentence list
  - Even slight variations count as different aliases (e.g., "customer" vs "the customer" are TWO separate aliases)
  - DO NOT merge aliases together - keep them distinct
  - Each alias should map ONLY to sentences where that EXACT form appears

  ### TASK:
  For each actor:
  1. Identify ALL distinct ways it is referenced (exact name, pronouns, role titles, variations)
  2. For EACH unique alias, list the sentence numbers where THAT SPECIFIC alias appears
  3. Treat each variation as a separate alias entry

  ### EXAMPLES:
  If "customer" appears in sentences 1, 3 and "the customer" appears in sentences 2, 4:
```
  aliases: [
    {{ alias: "customer", sentences: [1, 3] }},
    {{ alias: "the customer", sentences: [2, 4] }}
  ]
```

  If "he" appears in sentence 5 referring to customer:
```
  {{ alias: "he", sentences: [5] }}
```

  ### COREFERENCE RULES:
  - Track pronoun chains: if "he" in sentence 5 refers to "customer" in sentence 1, add "he" as a separate alias for the customer actor
  - Resolve possessive pronouns (his, her, their) to their antecedents
  - Identify role-based references (the admin, admin, administrator - all separate aliases)
  - Each demonstrative reference is a unique alias (this person, that user)

  ### OUTPUT REQUIREMENTS:
  - In the 'sentences' field, use ONLY the sentence number (integer starting from 1)
  - Each alias must appear as a separate AliasItem
  - Do NOT combine or merge similar aliases
  """
  
  response: ActorAliasMapping = structured_llm.invoke([
    ("system", system_prompt),
    ("human", user_prompt)
  ])
  
  
  # result = []
  
  # for item in response.mappings:
  #   item.sentences = [sentence[0] for sentence in item.sentences]
  #   print(item.sentences)
  # print(result)
  
  return {
    "actor_aliases": response.mappings
  }

In [104]:
#revalidate actors list

def actors_validate_node(state: GraphState):
  
  structured_llm = model.with_structured_output(ValidationResult)
  
  #input
  
  input_actors = state.get("actors")
  input_actors_alias = "\n".join([str(alias) for alias in state.get("actor_aliases")])
  input_text = state.get("input_text")
  
  system_prompt = """You are a data validation expert specializing in entity recognition and alias validation.

Your task is to validate actors (entities) and their aliases against the original raw input to ensure:
1. All actors mentioned in the raw input are captured
2. Aliases correctly refer to their corresponding actors
3. Sentence indices are valid and correctly reference sentences in the raw input
4. Aliases appear in the referenced sentences
5. No duplicate or conflicting aliases across different actors
6. Aliases are meaningful and contextually appropriate
7. All actors have complete information (name, aliases, sentence indices)

Note: Sentence indices start from 1 and reference the position of sentences in the raw input.

Return a structured validation result with:
- is_valid: boolean indicating if validation passed
- errors: list of specific validation errors found (critical issues)
- warnings: list of potential issues that don't fail validation
- suggestions: list of recommended improvements"""
  
  human_prompt = f"""Please validate the following actors and their aliases against the original raw input:

**Raw Input:**
{input_text}

**Actors List:**
{input_actors}

**Actor Aliases with Sentence Indices:**
{input_actors_alias}

Check for:
1. All actors from raw input are included in the actors list
2. Each actor has corresponding alias information
3. Sentence indices are valid (within range of sentences in raw input, starting from 1)
4. The actor or its aliases actually appear in the referenced sentences
5. No alias conflicts (same alias for different actors)
6. No duplicate aliases within the same actor
7. Sentence indices are correct and reference the right sentences
8. Format consistency across all ActorAlias entries
9. Completeness: each actor has at least name and sentence indices
10. No hallucinated actors or aliases not in raw input
11. No invalid sentence indices (e.g., index 0, negative numbers, or exceeding total sentence count)

Provide detailed validation results with specific examples of any issues found."""
  
  response: ValidationResult = structured_llm.invoke([
    ("system", system_prompt),
    ("human", human_prompt)
  ])
  
  print(response)
  
  return {
    "validation_actor_result": response
  }
  
def decide_after_actor_validation(state:GraphState):
  validation_result = state.get("validation_actor_result")
  if validation_result and validation_result.is_valid:
      return "valid"  # Tiếp tục flow bình thường
  else:
      return "invalid"  # Cần xử lý lại hoặc sửa lỗi

In [105]:
#define usecase node

def define_usecase_node(state: GraphState):
  
  
  
  return state

In [106]:
workflow = StateGraph(GraphState)

workflow.add_node("actors_node", actors_node)
workflow.add_node("actors_alias_node", actors_alias_node)
# workflow.add_node("actors_validate_node", actors_validate_node)


workflow.add_edge(START, "actors_node")
workflow.add_edge("actors_node", "actors_alias_node")
workflow.add_edge("actors_alias_node", END)

# workflow.add_edge("actors_alias_node", "actors_validate_node")
# workflow.add_conditional_edges(
#   "actors_validate_node",
#   decide_after_actor_validation,
#   {
#     "valid": END,
#     "invalid": "actors_node"
#   }
# )

app = workflow.compile()

In [107]:
with open("./input.txt", "r", encoding="UTF-8") as f:
  input = f.read()
  
input = preprocessing.normalize.whitespace(input)

nlp = spacy.load("en_core_web_lg")
doc = nlp(input)

chunks = extract.noun_chunks(doc, min_freq=1)
chunks = [chunk.text for chunk in chunks if chunk.root.pos_ != 'PRON' and chunk.root.lemma_.lower() not in INTERNAL_SYSTEM_KEYWORDS]

#Trạng thái khởi tạo ban đầu cho process
initial_state = {
  "input_text": input,
  "doc": doc,
  "actors": chunks,
  "actor_aliases": [],
  "usecases": [],
  "svo_elements": [],
}

result: GraphState = app.invoke(initial_state)

print("\n=== ACTORS ===")
print(result.get("actors"))
print("\n=== ACTOR ALIASES ===")
for item in result.get("actor_aliases"):
  print(item)


=== ACTORS ===
['librarian', 'student', 'automated kiosk', 'administrator']

=== ACTOR ALIASES ===


AttributeError: 'AliasItem' object has no attribute 'alias'